In [13]:
from torch import nn, tensor

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(3, 4)
        self.output = nn.Linear(4, 1)
        self.relu = nn.ReLU();

    def forward(self, x):
        output = self.hidden(x)
        output = self.relu(output)
        return self.output(output)

# Model description

## Layers:

This model has two layers:

1. A hidden layer with 4 neurons and a ReLU activation function.
2. An output layer with 1 neuron.

In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('static/car_data.csv', usecols=['combination_mpg', 'fuel_type', 'cylinders'])
label_encoder = LabelEncoder()
data['fuel_type'] = label_encoder.fit_transform(data['fuel_type'])

train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

train_data = tensor(train_data.values)

# Data

1. Load data using pands package, split into two clusters, half for training, half for testing my model after it.
2. Pick only useful columns
3. Parse from DataFrame to Tensor

## NOTE:
We must transform/cast all fields to digital type, so we can cast the DataFrame to a Tensor.
Why do all that ? We can only use tensors to train a model with torch. 